# Assignment 2

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the emnist_train.pkl file
with open('emnist_train.pkl', 'rb') as f:
    train_dict = pickle.load(f, encoding='bytes')

# Extract the data and labels arrays from the dictionary
train_data = train_dict['data']
train_labels = train_dict['labels']


# Load the emnist_test.pkl file
with open('emnist_test.pkl', 'rb') as f:
    test_dict = pickle.load(f, encoding='bytes')
    
# Extract the data and labels arrays from the dictionary
test_data = test_dict['data']
test_labels = test_dict['labels']

# Convert the data and labels arrays to NumPy arrays
train_data = np.array(train_data, dtype=np.float32)
train_labels = np.array(train_labels, dtype=np.int32)
test_data = np.array(test_data, dtype=np.float32)
test_labels = np.array(test_labels, dtype=np.int32)

# Define class names
# Create class names for all 62 classes
class_names = ['Class {}'.format(i) for i in range(62)] 
#Plot a grid of EMNIST examples of a specified size."""

def plot_examples(data, pred, target, n_rows=5, n_cols=5):
    """Plot a grid of EMNIST examples of a specified size."""
    # Size figure depending on the size of the grid
    plt.figure(figsize=(n_cols * 1.2, n_rows * 2.0))
    
    for row in range(n_rows):
        for col in range(n_cols):
            # Get the next index of the image
            index = n_cols * row + col
            
            # Reshape the flattened image to its original shape
            image = data[index].reshape((28, 28))
            
            # Plot the image at the appropriate place in the grid
            plt.subplot(n_rows, n_cols, index + 1)
            plt.imshow(image, cmap="binary")
            plt.axis('off')
            plt.title(class_names[target[index]] + '\n' + '>>' + class_names[pred[index]])
    
    plt.tight_layout()
    plt.show()

# Plot examples from different classes
fig, axes = plt.subplots(8, 8, figsize=(12, 12))
for i, ax in enumerate(axes.flat):
    if i < len(class_names):
        class_index = i  # Compute the class index
        # Get samples of the current class
        class_samples = train_data[train_labels == class_index]
        # Randomly select a sample from the class
        sample_index = np.random.randint(class_samples.shape[0])  
        ax.imshow(class_samples[sample_index].reshape((28, 28)), cmap='gray')
        ax.axis('off')
        # Set the title as the class name
        ax.set_title(class_names[class_index])  
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()

# MLP without Hyperparameter tuning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Reshape the data to 2D for MLP
train_data = train_data.reshape((-1, 28*28))
test_data = test_data.reshape((-1, 28*28))

# Normalize the data
train_data = train_data / 255.0
test_data = test_data / 255.0

# Convert the labels to one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Create the MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(28*28,)),
    Dense(64, activation='relu'),
    Dense(len(train_labels[0]), activation='softmax')  # Number of output classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Record the start time
start_time = time.time()

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1)

# Record the end time
end_time = time.time()

# Calculate and print the training time
train_time = end_time - start_time
print('Training time:', train_time)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Generate predictions
preds = model.predict(test_data)
preds_classes = np.argmax(preds, axis=1)

# Convert back to integer format
test_labels_int = np.argmax(test_labels, axis=1)

# Generate and print the classification report
classification_report_results = classification_report(test_labels_int, preds_classes)
print("Classification Report:")
print(classification_report_results)

# Generate and print the confusion matrix
confusion_matrix_results = confusion_matrix(test_labels_int, preds_classes)
print("Confusion Matrix:")
print(confusion_matrix_results)

accuracy = accuracy_score(test_labels_int, preds_classes)
precision = precision_score(test_labels_int, preds_classes, average='macro')
recall = recall_score(test_labels_int, preds_classes, average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# MLP with Hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Define the model building function required for KerasClassifier
def create_model(n_hidden_layers=2, n_hidden_neurons=50, activation_function='relu'):
    model = Sequential()
    model.add(Flatten(input_shape=(28*28,)))
    for _ in range(n_hidden_layers):
        model.add(Dense(n_hidden_neurons, activation=activation_function))
    model.add(Dense(len(train_labels[0]), activation='softmax'))  # Number of output classes
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Instantiate KerasClassifier with the new model
keras_classifier = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, verbose=0)
# Define the parameter grid
param_grid = {
    "n_hidden_neurons": [50, 100, 200],
    "activation_function": ["relu", "sigmoid", "tanh"]
}

# Instantiate GridSearchCV
grid = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, cv=3)

# Perform hyperparameter tuning / model selection
grid_result = grid.fit(train_data, train_labels)

# Print the best score and best parameters
print(grid_result.best_score_, grid_result.best_params_)

In [ ]:
# Evaluate the model on test data
test_loss, test_acc = grid_result.best_estimator_.model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Generate predictions
preds = grid_result.best_estimator_.model.predict(test_data)
preds_classes = np.argmax(preds, axis=1)

# Convert back to integer format
test_labels_int = np.argmax(test_labels, axis=1)

# Generate and print the classification report
classification_report_results = classification_report(test_labels_int, preds_classes)
print("Classification Report:")
print(classification_report_results)

# Generate and print the confusion matrix
confusion_matrix_results = confusion_matrix(test_labels_int, preds_classes)
print("Confusion Matrix:")
print(confusion_matrix_results)

accuracy = accuracy_score(test_labels_int, preds_classes)
precision = precision_score(test_labels_int, preds_classes, average='macro')
recall = recall_score(test_labels_int, preds_classes, average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# MLP Experiments with varying n_hidden_neurons_values and activation_function

In [ ]:
#vary n_hidden_neurons_values 10 trials with activation_function='relu'
n_hidden_neurons_values = np.linspace(50, 200, num=10, dtype=int)
n_iterations = 1 

best_score = 0
best_params = {}

for n_hidden_neurons in n_hidden_neurons_values:
    for i in range(n_iterations):
        start_time = time.time()
        
        model = create_model(n_hidden_neurons=n_hidden_neurons, activation_function='relu')
        history = model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1, verbose=0)

        # Generate predictions
        preds = model.predict(test_data)
        preds_classes = np.argmax(preds, axis=1)
        
        # Convert back to integer format
        test_labels_int = np.argmax(test_labels, axis=1)

        accuracy = accuracy_score(test_labels_int, preds_classes)
        precision = precision_score(test_labels_int, preds_classes, average='macro')
        recall = recall_score(test_labels_int, preds_classes, average='macro')

        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f'Iteration {i+1}, n_hidden_neurons: {n_hidden_neurons}')
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall)
        print('Elapsed time:', elapsed_time, 'seconds')

        if accuracy > best_score:
            best_score = accuracy
            best_params = {'n_hidden_neurons': n_hidden_neurons}

print('Best parameters:', best_params, 'with highest accuracy:', best_score)

In [ ]:
#vary n_hidden_neurons_values 10 trials with activation_function='sigmoid'
n_hidden_neurons_values = np.linspace(50, 200, num=10, dtype=int)
n_iterations = 1 

best_score = 0
best_params = {}

for n_hidden_neurons in n_hidden_neurons_values:
    for i in range(n_iterations):
        start_time = time.time()
        
        model = create_model(n_hidden_neurons=n_hidden_neurons, activation_function='sigmoid')
        history = model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1, verbose=0)

        # Generate predictions
        preds = model.predict(test_data)
        preds_classes = np.argmax(preds, axis=1)
        
        # Convert back to integer format
        test_labels_int = np.argmax(test_labels, axis=1)

        accuracy = accuracy_score(test_labels_int, preds_classes)
        precision = precision_score(test_labels_int, preds_classes, average='macro')
        recall = recall_score(test_labels_int, preds_classes, average='macro')

        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f'Iteration {i+1}, n_hidden_neurons: {n_hidden_neurons}')
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall)
        print('Elapsed time:', elapsed_time, 'seconds')

        if accuracy > best_score:
            best_score = accuracy
            best_params = {'n_hidden_neurons': n_hidden_neurons}

print('Best parameters:', best_params, 'with highest accuracy:', best_score)

In [ ]:
#vary n_hidden_neurons_values 10 trials with activation_function='tanh'
n_hidden_neurons_values = np.linspace(50, 200, num=10, dtype=int)
n_iterations = 1 

best_score = 0
best_params = {}

for n_hidden_neurons in n_hidden_neurons_values:
    for i in range(n_iterations):
        start_time = time.time()
        
        model = create_model(n_hidden_neurons=n_hidden_neurons, activation_function='tanh')
        history = model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1, verbose=0)

        # Generate predictions
        preds = model.predict(test_data)
        preds_classes = np.argmax(preds, axis=1)
        
        # Convert back to integer format
        test_labels_int = np.argmax(test_labels, axis=1)

        accuracy = accuracy_score(test_labels_int, preds_classes)
        precision = precision_score(test_labels_int, preds_classes, average='macro')
        recall = recall_score(test_labels_int, preds_classes, average='macro')

        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f'Iteration {i+1}, n_hidden_neurons: {n_hidden_neurons}')
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall)
        print('Elapsed time:', elapsed_time, 'seconds')

        if accuracy > best_score:
            best_score = accuracy
            best_params = {'n_hidden_neurons': n_hidden_neurons}

print('Best parameters:', best_params, 'with highest accuracy:', best_score)

# CNN before tuning 

rerun the first block and continue

In [ ]:
import numpy as np
import pickle
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Reshape the data for CNN
train_data = train_data.reshape((-1, 28, 28, 1))
test_data = test_data.reshape((-1, 28, 28, 1))

# Normalize the data
train_data = train_data / 255.0
test_data = test_data / 255.0

# Convert the labels to one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_labels[0]), activation='softmax')  # Number of output classes
])


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Record the start time
start_time = time.time()

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1)

# Record the end time
end_time = time.time()

# Calculate and print the training time
train_time = end_time - start_time
print('Training time:', train_time)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Generate predictions
preds = model.predict(test_data)
preds_classes = np.argmax(preds, axis=1)

# Convert back to integer format
test_labels_int = np.argmax(test_labels, axis=1)

# Generate and print the classification report
classification_report_results = classification_report(test_labels_int, preds_classes)
print("Classification Report:")
print(classification_report_results)

# Generate and print the confusion matrix
confusion_matrix_results = confusion_matrix(test_labels_int, preds_classes)
print("Confusion Matrix:")
print(confusion_matrix_results)

accuracy = accuracy_score(test_labels_int, preds_classes)
precision = precision_score(test_labels_int, preds_classes, average='macro')
recall = recall_score(test_labels_int, preds_classes, average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# CNN with Data Augmentation

In [ ]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image 
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
)
datagen.fit(train_data)

# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),  # Dropout regularization
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),  # Dropout regularization
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout regularization
    Dense(len(train_labels[0]), activation='softmax')  # Number of output classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Record the start time
start_time = time.time()

# Train the model
model.fit(datagen.flow(train_data, train_labels, batch_size=128), epochs=10, validation_data=(test_data, test_labels))

# Record the end time
end_time = time.time()

# Calculate and print the training time
train_time = end_time - start_time
print('Training time:', train_time)


# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Generate predictions
preds = model.predict(test_data)
preds_classes = np.argmax(preds, axis=1)

# Convert back to integer format
test_labels_int = np.argmax(test_labels, axis=1)

# Generate and print the classification report
classification_report_results = classification_report(test_labels_int, preds_classes)
print("Classification Report:")
print(classification_report_results)

# Generate and print the confusion matrix
confusion_matrix_results = confusion_matrix(test_labels_int, preds_classes)
print("Confusion Matrix:")
print(confusion_matrix_results)

accuracy = accuracy_score(test_labels_int, preds_classes)
precision = precision_score(test_labels_int, preds_classes, average='macro')
recall = recall_score(test_labels_int, preds_classes, average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# Added more Dropout layers and BatchNormalization layers, EarlyStopping callback, and narrowed the search range of learning rate

rerun the first block and continue

In [ ]:
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score

# Assuming train_data, test_data, train_labels, and test_labels are defined

# Reshape and normalize the data
train_data = train_data.reshape((-1, 28, 28, 1)) / 255.0
test_data = test_data.reshape((-1, 28, 28, 1)) / 255.0

# Convert the labels to one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Flatten(),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(len(train_labels[0]), activation='softmax')  # Number of output classes
])

# Set learning rate range
optimizer = Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Create callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('model.h5', save_best_only=True)

# Record the start time
start_time = time.time()

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1, 
          callbacks=[early_stopping, model_checkpoint])

# Record the end time
end_time = time.time()

# Calculate and print the training time
train_time = end_time - start_time
print('Training time:', train_time)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Generate predictions
preds = model.predict(test_data)
preds_classes = np.argmax(preds, axis=1)

# Convert back to integer format
test_labels_int = np.argmax(test_labels, axis=1)

# Generate and print the classification report
classification_report_results = classification_report(test_labels_int, preds_classes)
print("Classification Report:")
print(classification_report_results)

# Generate and print the confusion matrix
confusion_matrix_results = confusion_matrix(test_labels_int, preds_classes)
print("Confusion Matrix:")
print(confusion_matrix_results)

accuracy = accuracy_score(test_labels_int, preds_classes)
precision = precision_score(test_labels_int, preds_classes, average='macro')
recall = recall_score(test_labels_int, preds_classes, average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# CNN with tuning and solve zero_division

In [ ]:
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score

# Import Keras Tuner related functions
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Define the hyperparameter search space
hyperparameters = HyperParameters()
hyperparameters.Int('conv_1_filters', 32, 128, step=32)
hyperparameters.Float('dropout_1', 0.2, 0.5, step=0.1)
hyperparameters.Int('conv_2_filters', 64, 256, step=32)
hyperparameters.Float('dropout_2', 0.2, 0.5, step=0.1)
hyperparameters.Int('dense_units', 128, 512, step=64)
hyperparameters.Float('dropout_3', 0.2, 0.5, step=0.1)
hyperparameters.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

# Reshape and normalize the data
train_data = train_data.reshape((-1, 28, 28, 1)) / 255.0
test_data = test_data.reshape((-1, 28, 28, 1)) / 255.0

# Convert the labels to one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the function to build the model
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('conv_1_filters', 32, 128, step=32), (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1)))

    model.add(Conv2D(hp.Int('conv_2_filters', 64, 256, step=32), (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1)))

    model.add(Flatten())

    model.add(Dense(hp.Int('dense_units', 128, 512, step=64), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_3', 0.2, 0.5, step=0.1)))

    model.add(Dense(len(train_labels[0]), activation='softmax'))

    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
    metrics=['accuracy'])
    
    return model

# Create the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='my_model'
)

# Create callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('model.h5', save_best_only=True)

# Perform hyperparameter tuning
tuner.search(train_data, train_labels, epochs=10, validation_split=0.1, 
             callbacks=[early_stopping, model_checkpoint])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_hps)

# Build the model with the best hyperparameters
model = build_model(best_hps)

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1, 
          callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Generate predictions
preds = model.predict(test_data)
preds_classes = np.argmax(preds, axis=1)

# Convert back to integer format
test_labels_int = np.argmax(test_labels, axis=1)

# Generate and print the classification report
classification_report_results = classification_report(test_labels_int, preds_classes, zero_division=1.0)
print("Classification Report:")
print(classification_report_results)

# Generate and print the confusion matrix
confusion_matrix_results = confusion_matrix(test_labels_int, preds_classes)
print("Confusion Matrix:")
print(confusion_matrix_results)

# Calculate and print the accuracy, precision, and recall scores
accuracy = accuracy_score(test_labels_int, preds_classes)
precision = precision_score(test_labels_int, preds_classes, average='macro', zero_division=1.0)
recall = recall_score(test_labels_int, preds_classes, average='macro', zero_division=1.0)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)


# CNN Experiments with varying learning_rates and dense_units_options

rerun the first block and continue

In [ ]:
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score

# Assuming train_data, test_data, train_labels, and test_labels are defined

# Reshape and normalize the data
train_data = train_data.reshape((-1, 28, 28, 1)) / 255.0
test_data = test_data.reshape((-1, 28, 28, 1)) / 255.0

# Convert the labels to one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the function to build the model
def build_model(units, learning_rate):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(units, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(len(train_labels[0]), activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Define the parameter grid
dense_units_options = list(range(128, 512 + 1, 128))  # 10 values from 128 to 512
learning_rates = [1e-2, 1e-3, 1e-4, 1e-5]  # 4 values for learning rate

# Create callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('model.h5', save_best_only=True)

# Record results
results = []

# Perform manual hyperparameter tuning
for units in dense_units_options:
    for learning_rate in learning_rates:
        start_time = time.time()
        
        model = build_model(units, learning_rate)

        model.fit(train_data, train_labels, epochs=10, batch_size=128, validation_split=0.1, 
                  callbacks=[early_stopping, model_checkpoint])

        test_loss, test_acc = model.evaluate(test_data, test_labels)
        end_time = time.time()

        # Generate predictions
        preds = model.predict(test_data)
        preds_classes = np.argmax(preds, axis=1)

        # Convert back to integer format
        test_labels_int = np.argmax(test_labels, axis=1)

        accuracy = accuracy_score(test_labels_int, preds_classes)
        precision = precision_score(test_labels_int, preds_classes, average='macro', zero_division=1.0)
        recall = recall_score(test_labels_int, preds_classes, average='macro', zero_division=1.0)

        results.append({
            'learning_rate': learning_rate,
            'units': units,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'runtime': end_time - start_time
        })

        print("Learning rate:", learning_rate)
        print("Units:", units)
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("Runtime:", end_time - start_time)
        print()

# Print final results
for result in results:
    print("Learning rate:", result['learning_rate'])
    print("Units:", result['units'])
    print("Accuracy:", result['accuracy'])
    print("Precision:", result['precision'])
    print("Recall:", result['recall'])
    print("Runtime:", result['runtime'])
    print()

# Linear model

rerun the first block and continue

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
#Data preprocessing
# Normalise and reshape data
train_images = train_data / 255.0
test_images = test_data / 255.0

# Reshape the data from (28, 28) to (784,)
train_images = train_data.reshape(-1, 784)
test_images = test_data.reshape(-1, 784)

# Normalize the data
scaler = StandardScaler()
train_images = scaler.fit_transform(train_images)
test_images = scaler.transform(test_images)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
# Create an instance of the linear SVM model
svm_model = SVC(kernel='linear')

start = time.time()
# Train the model
svm_model.fit(X_train, y_train)

end = time.time()
print(end - start)

# Make predictions on the test set
predictions = svm_model.predict(X_test)

start = time.time()

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
confusion_mat = confusion_matrix(y_test, predictions)

end = time.time()
print(end - start)

# Print the evaluation metrics and confusion matrix
print("Accuracy_Linear:", accuracy)
print("Precision_Linear:", precision)
print("Recall_Linear:", recall)
print("Confusion Matrix_Linear:")
print(confusion_mat)

# PCA compression on Linear model

In [ ]:
# apply PCA without reducing dimensionality, then compute the min number of dimensions for preserving 95% variance
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1


# Apply PCA for dimensionality reduction

pca = PCA(n_components=d)
X_train_pca = pca.fit_transform(X_train)
# Transform the test set
X_test_pca = pca.transform(X_test)

# Fit the SVM model on the transformed training set
svm_model = SVC(kernel= 'linear')

start = time.time()

svm_model.fit(X_train_pca, y_train)

end = time.time()
print(end - start)

# Make predictions on the transformed test set
predictions_pca = svm_model.predict(X_test_pca)

start = time.time()
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions_pca)

end = time.time()
print(end - start)

print("Reduced data Accuracy:", accuracy)


# Plotting the reduced data to find if they form uniform clusters

plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('EMNIST ByClass Dataset - PCA')
plt.colorbar(label='Class')
plt.savefig('pca.pdf')
plt.show()

# Plot examples of PCA compressed SVM results
plot_examples(X_test, predictions_pca, y_test, n_rows=5, n_cols=5)

# SVM with default paramters

In [ ]:
# Create an instance of the SVM model with default hyperparameters
svm_model = SVC(kernel='poly', C=1.0, gamma='scale')

start = time.time()
# Train the model
svm_model.fit(X_train, y_train)
end = time.time()
print(end - start)

# Make predictions on the test set
predictions_SVMD = svm_model.predict(X_test)

start = time.time()

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, predictions_SVMD)
precision = precision_score(y_test, predictions_SVMD, average='macro')
recall = recall_score(y_test, predictions_SVMD, average='macro')
confusion_mat = confusion_matrix(y_test, predictions_SVMD)

end = time.time()
print(end - start)

# Print the evaluation metrics and confusion matrix
print("Accuracy_SVMD:", accuracy)
print("Precision_SVMD:", precision)
print("Recall_SVMD:", recall)
print("Confusion Matrix_SVMD:")
print(confusion_mat)

# SVM with hyperparamter tuning

In [ ]:
# Define the parameter grid
param_grid = {
    'C': [ 0.1, 1.0, 10, 100],
    'gamma': ['scale', 'auto']
}
# Create an instance of the SVM model
svm_model_param_grid = SVC(kernel='rbf')


# Split the data into training and testing sets for gridsearch
X_train_Grid, X_test_Grid, y_train_Grid, y_test_Grid = train_test_split(train_images, train_labels, test_size=0.5, random_state=42)

#Create an instance of the GridSearchCV with the SVM model and parameter grid
grid_search = GridSearchCV(svm_model_param_grid, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

start = time.time()
# Perform grid search on the training data
grid_search.fit(X_train_Grid, y_train_Grid)
end = time.time()
print(end - start)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
predictions_SVMBestparam = best_model.predict(X_test_Grid)

start = time.time()
# Calculate evaluation metrics
accuracy = accuracy_score(y_test_Grid, predictions_SVMBestparam)
precision = precision_score(y_test_Grid, predictions_SVMBestparam, average='macro')
recall = recall_score(y_test_Grid, predictions_SVMBestparam, average='macro')
confusion_mat = confusion_matrix(y_test_Grid, predictions_SVMBestparam)

end = time.time()
print(end - start)

# Print the evaluation metrics and confusion matrix
print("Best Parameters:", best_params)
print("Accuracy_SVMBestparam:", accuracy)
print("Precision_SVMBestparam:", precision)
print("Recall_SVMBestparam:", recall)
print("Confusion Matrix_SVMBestparam:")
print(confusion_mat)

# SVM with best parameters

In [ ]:
# Create an instance of the SVM model with the best hyperparameters
svm_model = SVC(kernel='rbf', C=10, gamma='auto')

start = time.time()
svm_model.fit(X_train, y_train)
end = time.time()
print(end - start)

# Make predictions
predictions_SVMNew = svm_model.predict(X_test)

start = time.time()
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, predictions_SVMNew)
precision = precision_score(y_test, predictions_SVMNew, average='macro')
recall = recall_score(y_test, predictions_SVMNew, average='macro')
confusion_mat = confusion_matrix(y_test, predictions_SVMNew)
end = time.time()
print(end - start)

# Print the evaluation metrics and confusion matrix
print("Accuracy_SVMNew:", accuracy)
print("Precision_SVMNew:", precision)
print("Recall_SVMNew:", recall)
print("Confusion Matrix_SVMNew:")
print(confusion_mat)

# Plot examples of best paramterized SVM
plot_examples(X_test, predictions_SVMNew, y_test, n_rows=3, n_cols=7)

# SVM with Regularization

In [ ]:
# Access the value of the C parameter
best_C = best_params['C']

# Apply regularization by reducing C value
regularized_C = best_C * 0.1  # Example: reduce C by 10 times

# Update the C parameter in the best_params dictionary
best_params['C'] = regularized_C

# Initialize a new SVM classifier with the regularized C value
regularized_svm = SVC(**best_params)

start = time.time()

# Fit the regularized SVM on the training data
regularized_svm.fit(X_train, y_train)

end = time.time()
print(end - start)

# Make predictions on the test set using the regularized model
predictions_Regularized = regularized_svm.predict(X_test)

start = time.time()
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, predictions_Regularized)
precision = precision_score(y_test, predictions_Regularized, average='macro')
recall = recall_score(y_test, predictions_Regularized, average='macro')
confusion_mat = confusion_matrix(y_test, predictions_Regularized)

end = time.time()
print(end - start)

# Print the evaluation metrics and confusion matrix
print("Regularized Parameters:", best_params)
print("Accuracy_Regularized:", accuracy)
print("Precision_Regularized:", precision)
print("Recall_Regularized:", recall)
print("Confusion Matrix_Regularized:")
print(confusion_mat)

# Plot examples of Regularized SVM
plot_examples(X_test, predictions_Regularized, y_test, n_rows=3, n_cols=7)

# SVM Experiments with varying C and Kernels

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np
import time

# Define parameter grid
C_values = np.logspace(-3, 2, num=5)  

# Define kernel types
kernels = ['rbf', 'poly']

# Split your data so that you only use 50% of it for training
sample_X_train, _, sample_y_train, _ = train_test_split(X_train, y_train, test_size=0.5, random_state=1)

best_score = 0
best_params = {}

# Perform grid search on the training data for each kernel type and 'C' value
for kernel in kernels:
    for C in C_values:
        start = time.time()

        # Create an instance of the SVM model with current kernel
        svm_model_param_grid = SVC(kernel=kernel, C=C)
        
        # Perform cross-validation on the training data
        scores = cross_val_score(svm_model_param_grid, sample_X_train, sample_y_train, cv=3)

        # Compute the mean accuracy and check if it's the best score so far
        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = {'kernel': kernel, 'C': C}

        # Fit the model with the 'C' parameter
        svm_model_param_grid.fit(sample_X_train, sample_y_train)

        # Make predictions on the test set using the model
        predictions_SVMBestparam = svm_model_param_grid.predict(X_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, predictions_SVMBestparam)
        precision = precision_score(y_test, predictions_SVMBestparam, average='macro')
        recall = recall_score(y_test, predictions_SVMBestparam, average='macro')
        confusion_mat = confusion_matrix(y_test, predictions_SVMBestparam)

        end = time.time()

        print("Parameters: Kernel=", kernel, "C=", C)
        print("Mean cross-validation accuracy:", mean_score)
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("Elapsed Time:", end - start)
        print()

print("Best Parameters:", best_params, "with highest mean cross-validation accuracy:", best_score)


# Graph process

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## SVM RESULTS

# Linear
l_c   = [0.0010, 100.00]

l_acc = [0.7264, 0.7264]
l_pre = [0.5819, 0.5819]
l_rec = [0.5627, 0.5627]
l_tim = [16.308, 16.308] # in mins

# non-linear
c     = [0.0010, 0.0178, 0.3162, 5.6234, 100.00]

# rbf
r_acc = [0.0547, 0.5330, 0.7315, 0.7877, 0.7774]
r_pre = [0.0078, 0.2271, 0.5956, 0.6843, 0.6745]
r_rec = [0.0162, 0.2063, 0.4716, 0.5996, 0.6010]
r_tim = [83.476, 371.14, 32.134, 25.899, 30.315] # in mins
     
# poly
p_acc = [0.0551, 0.2600, 0.6809, 0.7869, 0.7811]
p_pre = [0.0474, 0.3639, 0.5755, 0.6781, 0.6726]
p_rec = [0.0175, 0.1091, 0.3982, 0.5922, 0.6062]
p_tim = [148.38, 64.932, 45.097, 30.960, 29.848] # in mins

In [ ]:
## SVM GRAPHS

# Produce Line Plots

fig, axs = plt.subplots(2, 2, figsize=(15, 7.5))

# ACCURACY
axs[0, 0].plot(c, r_acc, color='cyan')
axs[0, 0].plot(c, p_acc, color='chartreuse')
axs[0, 0].plot(l_c, l_acc, color='orange', linestyle='dashed')
axs[0, 0].legend(('RBF','Polynomial','Linear'),loc='upper left')
axs[0, 0].set_title('SVM Accuracy')
#axs[0, 0].set_xlabel('C Value')
axs[0, 0].set_ylabel('Accuracy Score')
axs[0, 0].set_xscale('log')

# PRECISION
axs[0, 1].plot(c, r_pre, color='cyan')
axs[0, 1].plot(c, p_pre, color='chartreuse')
axs[0, 1].plot(l_c, l_pre, color='orange', linestyle='dashed')
#axs[0, 1].legend(('RBF','Polynomial','Linear'),loc='upper left')
axs[0, 1].set_title('SVM Precision')
#axs[0, 1].set_xlabel('C Value')
axs[0, 1].set_ylabel('Precision Score')
axs[0, 1].set_xscale('log')

# RECALL
axs[1, 0].plot(c, r_rec, color='cyan')
axs[1, 0].plot(c, p_rec, color='chartreuse')
axs[1, 0].plot(l_c, l_rec, color='orange', linestyle='dashed')
#axs[1, 0].legend(('RBF','Polynomial','Linear'),loc='upper left')
axs[1, 0].set_title('SVM Recall')
axs[1, 0].set_xlabel('C Value')
axs[1, 0].set_ylabel('Recall Score')
axs[1, 0].set_xscale('log')

# COMPUTATIONAL TIME
axs[1, 1].plot(c, r_tim, color='cyan')
axs[1, 1].plot(c, p_tim, color='chartreuse')
axs[1, 1].plot(l_c, l_tim, color='orange', linestyle='dashed')
#axs[1, 1].legend(('RBF','Polynomial','Linear'),loc='upper left')
axs[1, 1].set_title('SVM Computational Time')
axs[1, 1].set_xlabel('C Value')
axs[1, 1].set_ylabel('Time Taken [mins]')
axs[1, 1].set_xscale('log')

#plt.savefig('svm_experiment_singlelegend.pdf', bbox_inches='tight', pad_inches=0.3, dpi = 500) #save plot as pdf

In [ ]:
## CNN RESULTS

units = [128.00, 256.00, 384.00, 512.00]

# LR 1e-2
e2acc = [0.8477, 0.8506, 0.8467, 0.8453]
e2pre = [0.8043, 0.8134, 0.7905, 0.7943]
e2rec = [0.6988, 0.7079, 0.7131, 0.7128]
e2tim = [15.330, 16.511, 16.525, 18.301] # in mins
     
# LR 1e-3
e3acc = [0.8526, 0.8573, 0.8541, 0.8522]
e3pre = [0.7867, 0.7893, 0.7530, 0.7637]
e3rec = [0.7073, 0.7183, 0.7165, 0.7207]
e3tim = [15.259, 15.162, 16.406, 18.554] # in mins
    
# LR 1e-4
e4acc = [0.8430, 0.8464, 0.8433, 0.8498]
e4pre = [0.7861, 0.7865, 0.7891, 0.7788]
e4rec = [0.6870, 0.6944, 0.7015, 0.6996]
e4tim = [15.029, 15.188, 17.574, 17.782] # in mins
    
# LR 1e-5
e5acc = [0.7347, 0.7670, 0.7773, 0.7899]
e5pre = [0.7163, 0.6845, 0.6900, 0.7265]
e5rec = [0.4756, 0.5549, 0.5786, 0.5978]
e5tim = [14.759, 16.532, 18.571, 18.290] # in mins

In [ ]:
## CNN GRAPHS

# Produce Line Plots

fig, axs = plt.subplots(2, 2, figsize=(15, 7.5))

# ACCURACY
axs[0, 0].plot(units, e2acc, color='cyan')
axs[0, 0].plot(units, e3acc, color='chartreuse')
axs[0, 0].plot(units, e4acc, color='orange')
axs[0, 0].plot(units, e5acc, color='violet')
axs[0, 0].legend(('Learning rate: 1e-02','Learning rate: 1e-03','Learning rate: 1e-04','Learning rate: 1e-05'),bbox_to_anchor=(0.39,0.63),loc='center right')
axs[0, 0].set_title('CNN Accuracy')
#axs[0, 0].set_xlabel('Number of Dense Units')
axs[0, 0].set_ylabel('Accuracy Score')
#axs[0, 0].set_xscale('log')

# PRECISION
axs[0, 1].plot(units, e2pre, color='cyan')
axs[0, 1].plot(units, e3pre, color='chartreuse')
axs[0, 1].plot(units, e4pre, color='orange')
axs[0, 1].plot(units, e5pre, color='violet')
#axs[0, 1].legend(('Learning rate: 1e-02','Learning rate: 1e-03','Learning rate: 1e-04','Learning rate: 1e-05'),bbox_to_anchor=(0.39,0.63),loc='center right')
axs[0, 1].set_title('CNN Precision')
#axs[0, 1].set_xlabel('Number of Dense Units')
axs[0, 1].set_ylabel('Precision Score')
#axs[0, 1].set_xscale('log')

# RECALL
axs[1, 0].plot(units, e2rec, color='cyan')
axs[1, 0].plot(units, e3rec, color='chartreuse')
axs[1, 0].plot(units, e4rec, color='orange')
axs[1, 0].plot(units, e5rec, color='violet')
#axs[1, 0].legend(('Learning rate: 1e-02','Learning rate: 1e-03','Learning rate: 1e-04','Learning rate: 1e-05'),bbox_to_anchor=(0.39,0.63),loc='center right')
axs[1, 0].set_title('CNN Recall')
axs[1, 0].set_xlabel('Number of Dense Units')
axs[1, 0].set_ylabel('Recall Score')
#axs[1, 0].set_xscale('log')

# COMPUTATIONAL TIME
axs[1, 1].plot(units, e2tim, color='cyan')
axs[1, 1].plot(units, e3tim, color='chartreuse')
axs[1, 1].plot(units, e4tim, color='orange')
axs[1, 1].plot(units, e5tim, color='violet')
#axs[1, 1].legend(('Learning rate: 1e-02','Learning rate: 1e-03','Learning rate: 1e-04','Learning rate: 1e-05'),bbox_to_anchor=(0.39,0.63),loc='center right')
axs[1, 1].set_title('CNN Computational Time')
axs[1, 1].set_xlabel('Number of Dense Units')
axs[1, 1].set_ylabel('Time Taken [mins]')
#axs[1, 0].set_xscale('log')

#plt.savefig('cnn_experiment_singlelegend.pdf', bbox_inches='tight', pad_inches=0.3, dpi = 500) #save plot as pdf

In [ ]:
## MLP RESULTS

n_neu = [50.000, 66.000, 83.000, 100.00, 116.00, 133.00, 150.00, 166.00, 183.00, 200.00]

# RELU
r_acc = [0.7868, 0.7922, 0.8030, 0.8025, 0.8086, 0.8125, 0.8125, 0.8147, 0.8154, 0.8126]
r_pre = [0.6636, 0.6652, 0.6812, 0.6863, 0.6939, 0.6982, 0.7013, 0.6930, 0.6940, 0.7021]
r_rec = [0.6185, 0.6344, 0.6429, 0.6486, 0.6528, 0.6641, 0.6544, 0.6728, 0.6648, 0.6722]
r_tim = [0.2138, 0.2233, 0.2327, 0.2923, 0.3003, 0.4704, 0.5358, 0.4253, 0.4250, 0.4445] # in mins

# SIGMOID
s_acc = [0.7568, 0.7700, 0.7840, 0.7901, 0.7984, 0.8029, 0.8053, 0.8106, 0.8093, 0.8116]
s_pre = [0.6096, 0.6250, 0.6600, 0.6758, 0.6876, 0.6874, 0.6887, 0.7030, 0.6879, 0.6993]
s_rec = [0.5539, 0.5751, 0.5995, 0.6182, 0.6297, 0.6400, 0.6351, 0.6487, 0.6430, 0.6541]
s_tim = [0.2808, 0.2752, 0.2767, 0.3148, 0.3440, 0.3292, 0.3756, 0.3317, 0.3293, 0.5226] # in mins

# TANH
t_acc = [0.7815, 0.7937, 0.7999, 0.8033, 0.8076, 0.8150, 0.8069, 0.8122, 0.8155, 0.8084]
t_pre = [0.6459, 0.6639, 0.6921, 0.6831, 0.6873, 0.7001, 0.6904, 0.6900, 0.7004, 0.7131]
t_rec = [0.6050, 0.6251, 0.6362, 0.6559, 0.6567, 0.6618, 0.6547, 0.6677, 0.6664, 0.6643]
t_tim = [0.2080, 0.2269, 0.2690, 0.2566, 0.2838, 0.2688, 0.2873, 0.2582, 0.4052, 0.4425] # in mins

In [ ]:
## MLP GRAPHS

# Produce Line Plots

fig, axs = plt.subplots(2, 2, figsize=(15, 7.5))

# ACCURACY
axs[0, 0].plot(n_neu, r_acc, color='cyan')
axs[0, 0].plot(n_neu, s_acc, color='chartreuse')
axs[0, 0].plot(n_neu, t_acc, color='orange')
axs[0, 0].legend(('ReLU','Sigmoid','Tanh'),loc='upper left')
axs[0, 0].set_title('MLP Accuracy')
#axs[0, 0].set_xlabel('Number of Hidden Neurons')
axs[0, 0].set_ylabel('Accuracy Score')

# PRECISION
axs[0, 1].plot(n_neu, r_pre, color='cyan')
axs[0, 1].plot(n_neu, s_pre, color='chartreuse')
axs[0, 1].plot(n_neu, t_pre, color='orange')
#axs[0, 1].legend(('ReLU','Sigmoid','Tanh'),loc='upper left')
axs[0, 1].set_title('MLP Precision')
#axs[0, 1].set_xlabel('Number of Hidden Neurons')
axs[0, 1].set_ylabel('Precision Score')

# RECALL
axs[1, 0].plot(n_neu, r_rec, color='cyan')
axs[1, 0].plot(n_neu, s_rec, color='chartreuse')
axs[1, 0].plot(n_neu, t_rec, color='orange')
#axs[1, 0].legend(('ReLU','Sigmoid','Tanh'),loc='upper left')
axs[1, 0].set_title('MLP Recall')
axs[1, 0].set_xlabel('Number of Hidden Neurons')
axs[1, 0].set_ylabel('Recall Score')

# COMPUTATIONAL TIME
axs[1, 1].plot(n_neu, r_tim, color='cyan')
axs[1, 1].plot(n_neu, s_tim, color='chartreuse')
axs[1, 1].plot(n_neu, t_tim, color='orange')
#axs[1, 1].legend(('ReLU','Sigmoid','Tanh'),loc='upper left')
axs[1, 1].set_title('MLP Computational Time')
axs[1, 1].set_xlabel('Number of Hidden Neurons')
axs[1, 1].set_ylabel('Time Taken [mins]')

#plt.savefig('mlp_experiment_singlelegend.pdf', bbox_inches='tight', pad_inches=0.3, dpi = 500) #save plot as pdf

In [ ]:
# Overall best models

#      acc.    pre.    rec.    tim.   # time is % of 30 mins
svm = [0.8042, 0.7042, 0.6340, 0.7657]
cnn = [0.8522, 0.7637, 0.7207, 0.6185]
mlp = [0.8126, 0.7021, 0.6722, 0.0148]

In [ ]:
measures = ("Accuracy Score", "Precision Score", "Recall Score", "Time Taken [30mins]")
scores = {
    'Support Vector Machine': (0.804, 0.704, 0.634, 0.766),
    'Convolutional Neura': (0.852, 0.764, 0.721, 0.619),
    'MLP': (0.813, 0.702, 0.672, 0.015),
}

x = np.arange(len(measures))  # the label locations
width = 0.18  # the width of the bars
multiplier = 0
colors = ['cyan','chartreuse','orange']

fig, ax = plt.subplots(figsize=(12, 6))

for attribute, measurement in scores.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute, color=colors[multiplier])
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_title('Comparison of Evaluation Metrics for All Models')
ax.set_xticks(x + width, measures)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 1)


#plt.savefig('best_models.pdf', bbox_inches='tight', pad_inches=0.3, dpi = 500) #save plot as pdf